In [1]:
import warnings
warnings.filterwarnings('ignore')
import os
from helper import load_env
load_env()

import openai
import yaml
from crewai import Agent, Task, Crew
from typing import List, Union
from pydantic import BaseModel, Field
from IPython.display import display, Markdown


In [2]:
os.environ['OPENAI_MODEL_NAME'] = 'gpt-4o-mini'

In [14]:
# Define project inputs
project = 'KPI Dashboard Rollout'
industry = 'Enterprise Reporting'
project_objectives = 'Design and deploy five KPI dashboards'
cost_structure = {
  'business_analyst': 1000,  # cost per person-week in USD
  'ux_designer': 1200,
  'developer' : 1500,
  'data_engineer': 1400,
  'qa_engineer': 1100,
  'deployment_specialist': 1000,
  'project_manager': 1300,
  'program_manager': 1400

}

project_requirements = """
Note: Project management and program management roles are expected to operate at fractional capacity (e.g., 1–2 hours/day or 20–30% allocation) across all scenarios.

1. KPI Catalog Development
- Identify stakeholders and business owners.
- Gather KPI requirements.
- Define and document KPIs.
- Get business sign-off.

2. Dashboard Design
- Define business questions and personas.
- Create mockups/wireframes.
- Align with UX expectations.
- Get approvals.

3. Dashboard Development
- Connect to approved data sources.
- Implement KPIs and build visuals.
- Optimize performance.

4. QA & Testing
- Test dashboard functions and data.
- Conduct UAT.
- Iterate based on feedback.

5. Deployment & Rollout
- Set up access controls.
- Deploy to production.
- Provide user training.
- Set up feedback process.

6. Known Project Scale
- Total dashboards: 5
- Estimated KPIs: 50 total (10 per dashboard)
- Estimated Dimensions: 30 total
- Dashboard-wise breakdown (approx.):
  * Executive Summary: 25% KPIs, 15% dimensions
  * Finance: 20% KPIs, 25% dimensions
  * Operations: 25% KPIs, 20% dimensions
  * Sales & Marketing: 20% KPIs, 25% dimensions
  * Customer Support: 10% KPIs, 15% dimensions

7. Dashboard Complexity Estimates
- Simple Dashboards: 2
- Medium Dashboards: 2
- Complex Dashboards: 1
"""

# Display inputs
formatted_output = f"""
**Project Type:** {project}

**Project Objectives:** {project_objectives}

**Industry:** {industry}

**Cost Structure:**
{cost_structure}

**Project Requirements:**
{project_requirements}
"""
display(Markdown(formatted_output))

# Define inputs
inputs = {
  'project_type': project,
  'project_objectives': project_objectives,
  'industry': industry,
  'cost_structure': cost_structure,
  'project_requirements': project_requirements,
}



**Project Type:** KPI Dashboard Rollout

**Project Objectives:** Design and deploy five KPI dashboards

**Industry:** Enterprise Reporting

**Cost Structure:**
{'business_analyst': 1000, 'ux_designer': 1200, 'developer': 1500, 'data_engineer': 1400, 'qa_engineer': 1100, 'deployment_specialist': 1000, 'project_manager': 1300, 'program_manager': 1400}

**Project Requirements:**

Note: Project management and program management roles are expected to operate at fractional capacity (e.g., 1–2 hours/day or 20–30% allocation) across all scenarios.

1. KPI Catalog Development
- Identify stakeholders and business owners.
- Gather KPI requirements.
- Define and document KPIs.
- Get business sign-off.

2. Dashboard Design
- Define business questions and personas.
- Create mockups/wireframes.
- Align with UX expectations.
- Get approvals.

3. Dashboard Development
- Connect to approved data sources.
- Implement KPIs and build visuals.
- Optimize performance.

4. QA & Testing
- Test dashboard functions and data.
- Conduct UAT.
- Iterate based on feedback.

5. Deployment & Rollout
- Set up access controls.
- Deploy to production.
- Provide user training.
- Set up feedback process.

6. Known Project Scale
- Total dashboards: 5
- Estimated KPIs: 50 total (10 per dashboard)
- Estimated Dimensions: 30 total
- Dashboard-wise breakdown (approx.):
  * Executive Summary: 25% KPIs, 15% dimensions
  * Finance: 20% KPIs, 25% dimensions
  * Operations: 25% KPIs, 20% dimensions
  * Sales & Marketing: 20% KPIs, 25% dimensions
  * Customer Support: 10% KPIs, 15% dimensions

7. Dashboard Complexity Estimates
- Simple Dashboards: 2
- Medium Dashboards: 2
- Complex Dashboards: 1



In [3]:
##Agents & Task Config files
files = {
    'agents': '~/config/agents.yaml',
    'tasks': '~/config/tasks.yaml'
}

configs = {}
for config_type, file_path in files.items():
    with open(file_path, 'r') as file:
        configs[config_type] = yaml.safe_load(file)

agents_config = configs['agents']
tasks_config = configs['tasks']

In [4]:
agents_config

{'requirement_analysis_agent': {'role': 'Requirements Analyst',
  'goal': 'Estimate only the business analysis effort required to define the KPI catalog. Do not elaborate on dashboard design, development, or data engineering unless directly required for KPI scoping. Your focus is on stakeholder identification, data source mapping, and KPI documentation. Use the number of dashboards and the complexity structure provided in the requirements for efforts and resource estimations.\n',
  'backstory': 'You are a senior business analyst experienced in large-scale KPI strategy and data governance. Your job is to identify stakeholders, map enterprise data sources, and drive structured KPI definition processes. You work with domain leads and executive sponsors to ensure alignment with business objectives.\n',
  'verbose': True},
 'design_agent': {'role': 'UX Design Lead',
  'goal': 'Estimate only the design and UX effort needed to translate KPI requirements into user-friendly dashboards. Do not r

In [5]:
display(tasks_config)

{'requirement_analysis': {'description': 'Estimate the business analysis effort required to develop a KPI catalog. Use inputs like project_requirements and project_objectives to identify stakeholders, map data sources, and define documentation needs. Factor in number of dashboard count and break up of complexity levels provided in requirements.\n',
  'expected_output': 'Detailed breakdown of business analysis efforts for stakeholder mapping, data source review, and KPI documentation aligned with project objectives and complexity tiers.\n',
  'async_execution': False},
 'design': {'description': 'Estimate the UX and design effort needed to convert KPI requirements into usable dashboards. Base effort on user personas and the complexity of 5 dashboards (2 low, 2 medium, 1 high).\n',
  'expected_output': 'UX effort estimation including wireframes, visual layout plans, persona coverage, and feedback loops tailored to dashboard complexity.\n',
  'async_execution': False},
 'development': {'d

In [6]:
# Define custom output models
class TaskEffort(BaseModel):
    task_name: str = Field(..., description="Name of the task")
    estimated_time_hours: float = Field(..., description="Estimated total time for the task in hours")

class ResourceRequirement(BaseModel):
    role: str = Field(..., description="Type of resource, e.g., 'developer', 'designer'")
    count: int = Field(..., description="Number of people needed for this role")
    hours_per_day: float = Field(..., description="Working hours per day for each resource")

class TaskResourcePlan(BaseModel):
    task_name: str = Field(..., description="Name of the task")
    duration_days: float = Field(..., description="Estimated number of days to complete the task based on known resources")
    resources: List[ResourceRequirement] = Field(..., description="Resources required to deliver this task")

class Sprint(BaseModel):
    sprint_name: str = Field(..., description="Sprint identifier or name")
    start_day: int = Field(..., description="Day of project start when sprint begins")
    end_day: int = Field(..., description="Day of project timeline when sprint ends")
    task_names: List[str] = Field(..., description="Tasks scheduled in this sprint")

class Milestone(BaseModel):
    milestone_name: str = Field(..., description="Name of the milestone")
    tasks: List[str] = Field(..., description="List of task IDs associated with this milestone")

class ScenarioPlan(BaseModel):
    name: str = Field(..., description="Name of the delivery scenario")
    total_duration_days: float = Field(..., description="Total estimated project duration in days")
    estimated_cost: float = Field(..., description="Estimated cost for this delivery scenario")
    risk_summary: str = Field(..., description="Narrative summary of potential risks in this scenario")
    total_resources: int = Field(..., description="Total number of resources used in this scenario")
    number_of_squads: int = Field(..., description="Number of parallel squads deployed")
    efforts: List[TaskEffort] = Field(..., description="List of task effort estimates")
    resource_plans: List[TaskResourcePlan] = Field(..., description="List of resource plans by task")
    sprints: List[Sprint] = Field(..., description="Sprint schedule across the project timeline")
    milestones: List[Milestone] = Field(..., description="Milestones for this scenario")

class ProjectPlan(BaseModel):
    scenarios: List[ScenarioPlan] = Field(..., description="Different delivery strategy scenarios including cost, timeline, resources, and risk")


In [7]:
# Create Agents
requirement_analysis_agent = Agent(config=agents_config['requirement_analysis_agent'])
design_agent = Agent(config=agents_config['design_agent'])
development_agent = Agent(config=agents_config['development_agent'])
data_engineering_agent = Agent(config=agents_config['data_engineering_agent'])  # NEW
qa_agent = Agent(config=agents_config['qa_agent'])
deployment_agent = Agent(config=agents_config['deployment_agent'])
resource_estimator_agent = Agent(config=agents_config['resource_estimator_agent'])

# Create Tasks - domain agents estimate both effort and resource plan for their own tasks

requirement_analysis_task = Task(
    config=tasks_config['requirement_analysis'],
    agent=requirement_analysis_agent,
    output_pydantic=TaskResourcePlan
)

design_task = Task(
    config=tasks_config['design'],
    agent=design_agent,
    output_pydantic=TaskResourcePlan
)

development_task = Task(
    config=tasks_config['development'],
    agent=development_agent,
    output_pydantic=TaskResourcePlan
)

data_engineering_task = Task(  # NEW
    config=tasks_config['data_engineering'],
    agent=data_engineering_agent,
    output_pydantic=TaskResourcePlan
)

qa_task = Task(
    config=tasks_config['qa'],
    agent=qa_agent,
    output_pydantic=TaskResourcePlan
)

deployment_task_obj = Task(
    config=tasks_config['deployment'],
    agent=deployment_agent,
    output_pydantic=TaskResourcePlan
)

resource_estimation_task = Task(
    config=tasks_config['resource_estimation'],
    agent=resource_estimator_agent,
    output_pydantic=ProjectPlan,
    verbose=True
)

# Create Crew
crew = Crew(
    agents=[
        requirement_analysis_agent,
        design_agent,
        development_agent,
        data_engineering_agent,  # NEW
        qa_agent,
        deployment_agent,
        resource_estimator_agent
    ],
    tasks=[
        requirement_analysis_task,
        design_task,
        development_task,
        data_engineering_task,  # NEW
        qa_task,
        deployment_task_obj,
        resource_estimation_task
    ],
    verbose=True
)



**Project Type:** KPI Dashboard Rollout

**Project Objectives:** Design and deploy five KPI dashboards

**Industry:** Enterprise Reporting

**Cost Structure:**
{'business_analyst': 1000, 'ux_designer': 1200, 'developer': 1500, 'data_engineer': 1400, 'qa_engineer': 1100, 'deployment_specialist': 1000, 'project_manager': 1300, 'program_manager': 1400}

**Project Requirements:**

Note: Project management and program management roles are expected to operate at fractional capacity (e.g., 1–2 hours/day or 20–30% allocation) across all scenarios.

1. KPI Catalog Development
- Identify stakeholders and business owners.
- Gather KPI requirements.
- Define and document KPIs.
- Get business sign-off.

2. Dashboard Design
- Define business questions and personas.
- Create mockups/wireframes.
- Align with UX expectations.
- Get approvals.

3. Dashboard Development
- Connect to approved data sources.
- Implement KPIs and build visuals.
- Optimize performance.

4. QA & Testing
- Test dashboard functions and data.
- Conduct UAT.
- Iterate based on feedback.

5. Deployment & Rollout
- Set up access controls.
- Deploy to production.
- Provide user training.
- Set up feedback process.

6. Known Project Scale
- Total dashboards: 5
- Estimated KPIs: 50 total (10 per dashboard)
- Estimated Dimensions: 30 total
- Dashboard-wise breakdown (approx.):
  * Executive Summary: 25% KPIs, 15% dimensions
  * Finance: 20% KPIs, 25% dimensions
  * Operations: 25% KPIs, 20% dimensions
  * Sales & Marketing: 20% KPIs, 25% dimensions
  * Customer Support: 10% KPIs, 15% dimensions

7. Dashboard Complexity Estimates
- Simple Dashboards: 2
- Medium Dashboards: 2
- Complex Dashboards: 1



In [9]:
inputs

{'project_type': 'KPI Dashboard Rollout',
 'project_objectives': 'Design and deploy five KPI dashboards',
 'industry': 'Enterprise Reporting',
 'cost_structure': {'business_analyst': 1000,
  'ux_designer': 1200,
  'developer': 1500,
  'data_engineer': 1400,
  'qa_engineer': 1100,
  'deployment_specialist': 1000,
  'project_manager': 1300,
  'program_manager': 1400},
 'project_requirements': '\nNote: Project management and program management roles are expected to operate at fractional capacity (e.g., 1–2 hours/day or 20–30% allocation) across all scenarios.\n\n1. KPI Catalog Development\n- Identify stakeholders and business owners.\n- Gather KPI requirements.\n- Define and document KPIs.\n- Get business sign-off.\n\n2. Dashboard Design\n- Define business questions and personas.\n- Create mockups/wireframes.\n- Align with UX expectations.\n- Get approvals.\n\n3. Dashboard Development\n- Connect to approved data sources.\n- Implement KPIs and build visuals.\n- Optimize performance.\n\n4. 

In [10]:
# Run the crew
result = crew.kickoff(inputs)


# Agent: Requirements Analyst
## Task: Estimate the business analysis effort required to develop a KPI catalog. Use inputs like project_requirements and project_objectives to identify stakeholders, map data sources, and define documentation needs. Factor in number of dashboard count and break up of complexity levels provided in requirements.



# Agent: Requirements Analyst
## Final Answer: 
**Business Analysis Effort Estimation for KPI Catalog Development:**

**1. Understanding Project Requirements and Objectives:** 

- Review existing project documents to clarify business objectives related to the KPIs. This will ensure alignment with overall organizational goals. 

*Estimated Effort: 1 week*

**2. Stakeholder Identification:** 

- **Identify key stakeholders:**
  - Executive sponsors (e.g., C-suite executives, business unit leaders)
  - Domain leads (e.g., finance, sales, marketing, operations)
  - Data governance representatives
  - IT and data engineering teams

- **Engagement Act